In [4]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing= fetch_california_housing()
print(housing)


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

In [5]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [6]:
data['price']=housing.target
data.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611


In [7]:
from urllib.parse import urlparse
x=data.drop(columns=["price"])
y=data["price"]

In [9]:
##hyperparameter tuning using gridsearchcv

def h(xtrain,ytrain,params):
    rf=RandomForestRegressor()
    gs=GridSearchCV(estimator=rf,param_grid=params,cv=3,verbose=2,n_jobs=-1,scoring="neg_mean_squared_error")
    gs.fit(xtrain,ytrain)
    return gs

In [ ]:
"""
accuracy_score() is designed for classification, where the goal is to predict discrete class labels (e.g., "cat", "dog", or 0, 1).

It calculates the proportion of correct predictions, i.e. how often ypred == ytrue.

🛑 But in regression, your model predicts continuous numeric values (e.g., prices, temperatures, scores), not class labels.

So using accuracy_score() in regression doesn't make sense — it would almost always be 0, because predicted values rarely match the true values exactly.

"""

In [11]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20)

from mlflow.models import infer_signature
sign=infer_signature(xtrain,ytrain)

params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

## mlflow

with mlflow.start_run():

    gs=h(xtrain,ytrain,params)
    ##find best model

    bs=gs.best_estimator_
    
    #eval the best model
    ypred=bs.predict(xtest)
    mse=mean_squared_error(ytest,ypred)

    #log the params metrics
mlflow.log_param("best n_estimators", gs.best_params_['n_estimators'])
mlflow.log_param("best max_depth", gs.best_params_['max_depth'])
mlflow.log_param("best min_samples_split", gs.best_params_['min_samples_split'])
mlflow.log_param("best min_samples_leaf", gs.best_params_['min_samples_leaf'])
mlflow.log_metric("mse",mse)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

if tracking_url_type_store != "file":
    mlflow.sklearn.log_model(bs,"model",registered_model_name="best randomforest model")
else:
    mlflow.sklearn.log_model(bs,"model",signature=sign)

print(f"best hyperparams:{gs.best_params_}")    
print(f"mean squared error: {mse}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/08/21 15:59:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 15:59:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'best randomforest model'.
2025/08/21 15:59:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best randomforest model, version 1
Created version '1' of model 'best randomforest model'.


best hyperparams:{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
mean squared error: 0.2631498966306956
